In [1]:
#Qmak_PrimaryCareFDM_FULL_20221201.ipynb
#Builds the PrimaryCare FDM
#Assumes that the targetdb below exists
#Needs code to build it if it doesn't

#You will need to manually amend the target dataset for this script

library(bigrquery)

# Store the project ID
project_id = "yhcr-prd-phm-bia-core"

sourcedb <-"yhcr-prd-phm-bia-core.CY_FDM_PrimaryCare_Staging_V6"
targetdb <-'yhcr-prd-phm-bia-core.CY_FDM_PrimaryCare_V6'
targetdb <-gsub(' ','',targetdb)
print (targetdb)

[1] "yhcr-prd-phm-bia-core.CY_FDM_PrimaryCare_V6"


In [ ]:
# Qmak_PrimaryCareCY_FDM_PrimaryCare_V6_Person_Table
# Last modified 20220831 - JDB
# Last run 20220920 
# file location - \\bhts-conydevwd2\d$\JDB\Scripts\CY_FDM_PrimaryCare

#build care sites just for PrimaryCare 

sql1 <-paste('drop table if exists ',targetdb,'.care_site', sep = "")
sql2<- paste('drop table if exists ' ,targetdb,'.tmp_orgs', sep = "")
sql3<- paste('drop table if exists ' ,targetdb,'.tmp_orgs2', sep = "")

#make the temp org tables 
sql4 <-paste('create table ' ,targetdb,'.tmp_orgs as select distinct IDOrganisation from ',sourcedb,'.tbl_SRCode', sep = "")
sql5 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationDoneAt from ',sourcedb,'.tbl_SRCode', sep = "")
#print (sql5)
sql6 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationRegisteredAt from ',sourcedb,'.tbl_SRCode', sep = "")
sql7 <-paste('insert into ' ,targetdb,'.tmp_orgs select distinct IDOrganisationVisibleTo from ',sourcedb,'.tbl_SRCode', sep = "")
sql8 <-paste('create table ' ,targetdb,'.tmp_orgs2 as Select distinct IDOrganisation from ' ,targetdb,'.tmp_orgs', sep = "")
sql9 <-paste('create table ' ,targetdb,'.care_site as select distinct a.* from yhcr-prd-phm-bia-core.CY_FDM_MASTER.care_site a ,',targetdb,'.tmp_orgs2 b where a.care_site_source_value = b.IDOrganisation and a.place_of_service_source_value = \'GP practice site\'', sep = "")
#print (sql9)
sql10 <-paste('delete from ' ,targetdb,'.care_site where care_site_id = 236 or care_site_id = 886' , sep = "")

#Now Run them 
tb1 <- bq_project_query(project_id, sql1)
tb2 <- bq_project_query(project_id, sql2)
tb3 <- bq_project_query(project_id, sql3)
tb4 <- bq_project_query(project_id, sql4)
tb5 <- bq_project_query(project_id, sql5)
tb6 <- bq_project_query(project_id, sql6)
tb7 <- bq_project_query(project_id, sql7)
tb8 <- bq_project_query(project_id, sql8)
tb9 <- bq_project_query(project_id, sql9)
tb10 <- bq_project_query(project_id, sql10)


#Now build the persons table 

sql11 <-paste('drop table if exists ' ,targetdb,'.person', sep = "")
tb11 <- bq_project_query(project_id, sql11)


sql12 <-paste('create table ' ,targetdb,'.person 
as 
SELECT distinct a.person_id
, gender_concept_id
, year_of_birth
, month_of_birth
, day_of_birth
, birth_datetime
, death_datetime
, race_concept_id
, ethnicity_concept_id
, location_id
, provider_id
, care_site_id
, person_source_value
, gender_source_value
, gender_source_concept_id
, race_source_value
, race_source_concept_id
, ethnicity_source_value
, ethnicity_source_concept_id 
FROM yhcr-prd-phm-bia-core.CY_FDM_MASTER.person a
,',sourcedb,'.tbl_SRCode b
where a.person_id =  b.person_id' , sep = "")

tb12 <- bq_project_query(project_id, sql12)


#Ensure you have updated the lookup table ie

#This will automatically update the extract date with the max(DateEventRcorded) from SRCode 
#as that is the end date.
sql13 <- paste('update yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_Dataset_ExtractDateRef set extract_date = (select cast(max(DateEventRecorded) as date) 
from ',sourcedb,'.tbl_SRCode) where DatasetName = \'PrimaryCare\'', sep = "")

#print (sql13)

tb13 <- bq_project_query(project_id, sql13)

#This will manually update the same record

#sql14 <- "update `yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_Dataset_ExtractDateRef` set extract_date = '2022-06-29' where DatasetName = 'PrimaryCare'"  

#tb14 <- bq_project_query(project_id, sql14)



#This query makes the observation period based on the persons table BUT adds in rules for this data extract.
#This query makes the observation period based on the persons table BUT adds in rules for this data extract.

#Qmak_PrimaryCareObservationPeriod_Part1

sql15 <-paste('drop table if exists ' ,sourcedb,'.tmp_Eventdates' , sep = "")
sql16 <-paste('drop table if exists ' ,sourcedb,'.tmp_EventdatesValid' , sep = "")
sql17 <-paste('truncate table ' ,targetdb,'.observation_period' , sep = "")

tb15 <-bq_project_query(project_id, sql15)
tb16 <-bq_project_query(project_id, sql16)
tb17 <-bq_project_query(project_id, sql17)


sql18 <-paste('create table ',sourcedb,'.tmp_Eventdates as 
select a.person_id 
, a.DateEvent as EventDate
from ',sourcedb,'.tbl_SRCode a ,' ,targetdb
,'.person e 
where e.person_id = a.person_id' , sep = "")


tb18 <-bq_project_query(project_id, sql18)



sql19 <-paste('insert into ',sourcedb,'.tmp_Eventdates
select a.person_id
, a.DateEvent as EventDate from ',sourcedb,'.tbl_SRPrimaryCareMedication a
,' ,targetdb,'.person e 
where e.person_id = a.person_id'  , sep = "")

sql20 <-paste('insert into ',sourcedb,'.tmp_Eventdates
select a.person_id
, a.DateEvent as EventDate from ',sourcedb,'.tbl_SRReferralIn a
,' ,targetdb,'.person e 
where e.person_id = a.person_id'  , sep = "")


sql21 <-paste('insert into ',sourcedb,'.tmp_Eventdates
select a.person_id
, a.DateEvent as EventDate from ',sourcedb,'.tbl_SRReferralOut a
,' ,targetdb,'.person e 
where e.person_id = a.person_id'  , sep = "")

sql22 <-paste('insert into ',sourcedb,'.tmp_Eventdates
select a.person_id
, a.DateBooked as EventDate from ',sourcedb,'.tbl_SRVisit a
,' ,targetdb,'.person e 
where e.person_id = a.person_id'  , sep = "") 

sql23 <-paste('insert into ',sourcedb,'.tmp_Eventdates
select distinct a.person_id
, cast(left(a.DateAppointmentBooked,10)  as date)  as EventDate from ',sourcedb,'.tbl_SRAppointment a
,' ,targetdb,'.person e 
where e.person_id = a.person_id' , sep = "") 

tb19 <-bq_project_query(project_id, sql19)
tb20 <-bq_project_query(project_id, sql20)
tb21 <-bq_project_query(project_id, sql21)
tb22 <-bq_project_query(project_id, sql22)
tb23 <-bq_project_query(project_id, sql23)

sql24 <-paste('create table ',sourcedb,'.tmp_EventdatesValid
as
select a.person_id
,cast(a.EventDate as date) as EventDate
from ',sourcedb,'.tmp_Eventdates a ,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is not null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= date_add(e.death_datetime, INTERVAL 42 day) 
and a.EventDate <= (Select extract_date from yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_Dataset_ExtractDateRef where DatasetName = \'PrimaryCare\')', sep = "")


#-- BUT this only includes persons with a death datetime
#-- So section below includes those with a null deathdatetime

sql25 <-paste('insert into ',sourcedb,'.tmp_EventdatesValid
select distinct a.person_id
, cast(a.EventDate as date) as EventDate
from ',sourcedb,'.tmp_Eventdates a
,' ,targetdb,'.person e 
where e.person_id = a.person_id 
and e.death_datetime is null 
and a.EventDate >= e.birth_datetime
and a.EventDate <= (Select extract_date from yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_Dataset_ExtractDateRef where DatasetName = \'PrimaryCare\')', sep = "")

#third part pushes these into the observation_period table

sql26 <-paste('insert into ' ,targetdb,'.observation_period
select distinct
ROW_NUMBER() over (Order by person_id) as observation_period_id 
, person_id
, min(EventDate) as observation_period_start_date
, max(EventDate) as observation_period_end_date
,null as period_type_concept_id
from ',sourcedb,'.tmp_EventdatesValid
group by person_id', sep = "")

#Finally Run these sql's


tb24 <-bq_project_query(project_id, sql24)
tb25 <-bq_project_query(project_id, sql25)
tb26 <-bq_project_query(project_id, sql26)

#Qapp_PrimaryCareFDM_SRAppointment
#Last Modified 20220831

sql27 <-paste('truncate table ' ,targetdb,'.tbl_SRAppointment', sep = "")

sql28 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DateAppointmentBooked  =  null where DateAppointmentBooked = \'NULL\'', sep = "")

sql29 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DatePatientArrival =  null where DatePatientArrival  =  \'NULL\'', sep = "")

sql30 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DatePatientSeen  = null where DatePatientSeen = \'NULL\'', sep = "")

sql31 <-paste('Update ',sourcedb,'.tbl_SRAppointment
Set DateAppointmentCancelled = null where DateAppointmentCancelled = \'NULL\'', sep = "")

sql32 <-paste(' insert into ' ,targetdb,'.tbl_SRAppointment 
SELECT distinct sro.person_id  
, sro.RowIdentifier  
, IDOrganisationVisibleTo
, DateStart
, DateEnd
, AppointmentStatus
, cast(DateAppointmentBooked as datetime) as DateAppointmentBooked
, cast(DatePatientArrival as datetime) as DatePatientArrival
, cast(DatePatientSeen as datetime) as DatePatientSeen
, FollowUpAppointment
, TelephoneAppointment
, IDClinician
, IDProfileClinician
, RotaName
, RotaType
, RotaLocation
, RotaCode
, IDRotaOwner
, IDProfileRotaOwner
, AllowsOverbooking
, BookingContactNumber
, cast(DateAppointmentCancelled as datetime) as DateAppointmentCancelled
, IDRota
, IDReferralIN
, IDPatient
, IDOrganisation
, IDOrganisationRegisteredAt
, cs3.care_site_id as care_site_id_VisibleTo
, cs1.care_site_id 
, cs2.care_site_id as care_site_RegisteredAt
, 0 as provider_id
#, cast(sro.provider_id as int64) as provider_id
FROM ', sourcedb,'.tbl_SRAppointment sro
, ' ,targetdb,'.observation_period obs
left join ' ,targetdb,'.care_site cs1
on sro.IDOrganisation = cs1.care_site_source_value
left join ' ,targetdb,'.care_site cs2
on sro.IDOrganisationRegisteredAt = cs2.care_site_source_value
left join ' ,targetdb,'.care_site cs3
on sro.IDOrganisationVisibleTo = cs3.care_site_source_value
where sro.person_id  = obs.person_id
and cast(sro.DateAppointmentBooked as datetime)  <= obs.observation_period_end_date
and cast(sro.DateAppointmentBooked as datetime)  >= obs.observation_period_start_date', sep = "")

#Now Run these 
tb27 <-bq_project_query(project_id, sql27)
tb28 <-bq_project_query(project_id, sql28)
tb29 <-bq_project_query(project_id, sql29)
tb30 <-bq_project_query(project_id, sql30)
tb31 <-bq_project_query(project_id, sql31)
tb32 <-bq_project_query(project_id, sql32)

#Qapp_PrimaryCareFDM_SRCode
#Last Modified 20220831
sql33 <-paste('truncate table ' ,targetdb,'.tbl_SRCode', sep = "")

tb33 <-bq_project_query(project_id, sql33)

sql34 <-paste('insert into ' ,targetdb,'.tbl_SRCode
(
person_id
, src_RowIdentifier
, src_dateeventrecorded
, src_dateevent
, src_iddoneby
, src_idorganisationdoneat
, src_ctv3code
, src_ctv3text
, src_snomedcode
, src_numericvalue
, src_numericunit
, src_idorganisation
, src_idorganisationregisteredat
, src_idevent
, src_idreferralin
, src_idappointment
, src_idvisit
,care_site_id
,care_site_id_DoneAt
,care_site_id_RegisteredAt
,procedure_source_concept_id
,observation_source_concept_id
,condition_source_concept_id
,measurement_source_concept_id
)
SELECT distinct src.person_id 
, src.rowidentifier 
, DateEventRecorded
, DateEvent
, cast(iddoneby as string) as IDDoneBy 
, idorganisationdoneat
, src.ctv3code
, ctv.CTV3Desc
, snm.SNOMEDCode
, cast(numericvalue as string) as numericvalue
, numericunit
, idorganisation
, idorganisationregisteredat
, cast(idevent as string) as idevent
, cast(idreferralin as string) as idreferralin
, cast(idappointment as string) as idappointment
, cast(idvisit as string) as idvisit
, cs1.care_site_id  as care_site_id
, cs3.care_site_id as care_site_DoneAt
, cs2.care_site_id as care_site_RegisteredAt
, con1.concept_id as procedure_source_concept_id
, con2.concept_id as observation_source_concept_id
, con3.concept_id as condition_source_concept_id
, con4.concept_id as measurement_source_concept_id
FROM ',sourcedb,'.tbl_SRCode src
, ' ,targetdb,'.observation_period obs
left join yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_CTV3Codes_Lookup ctv on src.ctv3code = ctv.CTV3Code
left join yhcr-prd-phm-bia-core.CY_LOOKUPS.tbl_CTV3ToSnomed_Map snm on src.ctv3code = snm.CTV3Code
left join ' ,targetdb,'.care_site cs1 on src.IDOrganisation = cs1.care_site_source_value
left join ' ,targetdb,'.care_site cs2 on src.IDOrganisationRegisteredAt = cs2.care_site_source_value
left join ' ,targetdb,'.care_site cs3 on src.IDOrganisationDoneAt = cs3.care_site_source_value
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con1 on snm.SNOMEDCode = con1.concept_code and ((con1.domain_id like \'Procedure%\' or con1.domain_id like \'Spec Anatomic%\') and con1.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr1 ON con1.concept_id = cr1.concept_id_1 and (cr1.relationship_id = \'Maps to\'and cr1.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con2 on snm.SNOMEDCode = con2.concept_code and (con2.domain_id like \'Observation%\' and con2.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr2 ON con2.concept_id = cr2.concept_id_1 and (cr2.relationship_id = \'Maps to\' and cr2.invalid_reason IS NULL )
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con3 on snm.SNOMEDCode = con3.concept_code  and ( con3.domain_id like \'Condition%\' and con3.vocabulary_id = \'SNOMED\') 
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr3 ON con3.concept_id = cr3.concept_id_1 and (cr3.relationship_id = \'Maps to\' and cr3.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con4 on snm.SNOMEDCode = con4.concept_code  and ( con4.domain_id like \'Measu%\' and con4.vocabulary_id = \'SNOMED\')
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr4 ON con4.concept_id = cr4.concept_id_1 and (cr4.relationship_id = \'Maps to\' and cr4.invalid_reason IS NULL)
where  src.person_id  = obs.person_id
and DateEvent  <= obs.observation_period_end_date
and DateEvent  >= obs.observation_period_start_date ', sep = "")

#Note: Snomed and ctv3code updated using left join on the Lookup tables in the lookup datasets

tb34 <-bq_project_query(project_id, sql34)

#Qapp_PrimaryCareFDM_SRImmunisation_V6
#Last Modified 20220831

sql35 <-paste('Update ',sourcedb,'.tbl_SRImmunisation
set dateexpiry = null where dateexpiry =\'NULL\'', sep = "")

sql36 <-paste('truncate table ' ,targetdb,'.tbl_SRImmunisation', sep = "")

sql37 <-paste('insert into  ' ,targetdb,'.tbl_SRImmunisation
SELECT distinct sri. person_id
, rowidentifier 
, idorganisationvisibleto
,  dateeventrecorded
,  dateevent
, idprofileenteredby
, iddoneby
, idorganisationdoneat
, idvaccination
, idimmunisationcontent
, dose
, location
, method
, cast(left(dateexpiry,19) as datetime) as dateexpiry
, immsreadcode
, vaccpart
, vaccbatchnumber
, vaccareacode
, vaccinationstatus
, idevent
, idpatient
, idorganisation
, idorganisationregisteredat 
, null as care_site_id
, null as care_site_Id_DoneAt
, null as care_site_id_RegisteredAt
FROM ',sourcedb,'.tbl_SRImmunisation sri
, ' ,targetdb,'.observation_period obs
where sri.person_id = obs.person_id
and DateEvent <= obs.observation_period_end_date
and DateEvent >= obs.observation_period_start_date', sep = "")

tb35 <-bq_project_query(project_id, sql35)
tb36 <-bq_project_query(project_id, sql36)
tb37 <-bq_project_query(project_id, sql37)


#now do immunisationcontent
sql38 <-paste('truncate table ' ,targetdb,'.tbl_SRImmunisationContent', sep = "")

sql39 <-paste('insert into ' ,targetdb,'.tbl_SRImmunisationContent
SELECT RowIdentifier,Name,Content,
DateDeleted
FROM ',sourcedb,'.tbl_SRImmunisationContent', sep = "")



tb38 <-bq_project_query(project_id, sql38)
tb39 <-bq_project_query(project_id, sql39)

#Qapp_PrimaryCareFDM_SRPatient_V6
#Last Modified 20220801

sql40 <-paste('truncate table ' ,targetdb,'.tbl_SRPatient', sep = "")



sql41 <-paste('insert into ' ,targetdb,'.tbl_SRPatient
(
 src_rowidentifier
,src_idorganisationvisibleto
,src_datebirth
,src_datedeath
,src_birthplace
,src_gender
,src_speaksenglish
,src_ssref
,src_spinematched
,src_ethnicity
,src_languagespoken
,src_soa
,src_ward
, person_id
#, care_site_id_visibleto
)
SELECT  RowIdentifier 
, IDOrganisationVisibleTo
, DateBirth
, DateDeath
, BirthPlace
, Gender
, SpeaksEnglish
, SSRef
, SpineMatched
, Ethnicity
, LanguageSpoken
, SOA
, Ward
,  person_id 
#, cs1.care_site_id as care_site_id_VisibleTo
FROM ',sourcedb,'.tbl_SRPatient srp', sep = "")

#Qapp_PrimaryCareFDM_SRPatientAddressHistory
#Last Modified 20220831

sql42 <-paste(' truncate table ' ,targetdb,'.tbl_SRPatientAddressHistory', sep = "")

sql43 <-paste(' insert into ' ,targetdb,'.tbl_SRPatientAddressHistory
SELECT  RowIdentifier
, IDOrganisationVisibleTo
, DateEventRecorded
, null as DateEvent
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, PartialPostCode
, DateTo
, AddressType
, IDEvent
, IDPatient
, person_id
, IDOrganisation
, CcgOfResidence
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_site_id_VisibleTo
 ,cs2.care_site_id  as care_site_id_DoneAt
 ,cs3.care_site_id  as care_site_id
 ,cs4.care_site_id  as  care_site_id_RegisteredAt
 ,null as provider_id
 #, cast(provider_id as int64) as prpovid
 FROM ',sourcedb,'.tbl_SRPatientAddressHistory src
 left join ' ,targetdb,'.care_site cs1
 on src.IDOrganisationVisibleTo = cs1.care_site_source_value
 left join ' ,targetdb,'.care_site cs2
 on src.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on src.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on src.IDOrganisationRegisteredAt = cs4.care_site_source_value', sep = "")

#Qapp_PrimaryCareFDM_SRPatientRegistration
#Last modified 20220318 - JDB

sql44 <-paste('Update ',sourcedb,'.tbl_SRPatientRegistration
set DateRegistration = null where DateRegistration = \'NULL\'', sep = "")

sql45 <-paste('Update ',sourcedb,'.tbl_SRPatientRegistration
set DateDeRegistration = null where DateDeRegistration = \'NULL\'', sep = "")

sql46 <-paste(' truncate table ' ,targetdb,'.tbl_SRPatientRegistration', sep = "")

sql47 <-paste(' insert into ' ,targetdb,'.tbl_SRPatientRegistration
SELECT  RowIdentifier
, IDOrganisationVisibleTo
, dateeventrecorded
, IDProfileEnteredBy
, cast(left(DateRegistration,19) as datetime) as DateRegistration
, cast(left(DateDeRegistration,19) as datetime) as DateDeRegistration
, RegistrationStatus
, PreferredPharmacy
, cast(IDPatient as int64) as IDPatient
, IDOrganisation
, IDOrganisationRegisteredAt
, cast(person_id as int64) as person_id
, null as care_site_id_VisibleTo
, null as care_side_id
, null as care_site_id_RegisteredAt
FROM ',sourcedb,'.tbl_SRPatientRegistration', sep = "")

tb40 <-bq_project_query(project_id, sql40)
tb41 <-bq_project_query(project_id, sql41)
tb42 <-bq_project_query(project_id, sql42)
tb43 <-bq_project_query(project_id, sql43)
tb44 <-bq_project_query(project_id, sql44)
tb45 <-bq_project_query(project_id, sql45)
tb46 <-bq_project_query(project_id, sql46)
tb47 <-bq_project_query(project_id, sql47)

#Qapp_PrimaryCareFDM_SRPrimaryCareMedication
#Last Modified 20220831 - JDB 

sql48 <-paste(' truncate table ' ,targetdb,'.tbl_SRPrimaryCareMedication', sep = "")

sql49 <-paste(' insert into ' ,targetdb,'.tbl_SRPrimaryCareMedication 
(
person_id
   , src_rowidentifier 
, src_idorganisationvisibleto
, src_dateeventrecorded
, src_dateevent
, src_iddoneby
, src_idorganisationdoneat
, src_idmultilexproduct
, src_idmultilexpack
, src_idmultilexdmd
, src_nameofmedication
, src_datemedicationstart
, src_datemedicationend
, src_medicationdosage
, src_medicationquantity
, src_issupervised
, src_superviseddose
, src_unsuperviseddose
, src_isrepeatmedication
, src_isothermedication
, src_isdentalmedication
, src_ishospitalmedication
, src_idorganisationregisteredat 
, care_site_id_VisibleTo   
, care_site_id_DoneAt
, care_site_id_RegisteredAt
#,drug_source_concept_id
#,device_source_concept_id
)
SELECT distinct srm.person_id
, cast(rowidentifier as string) as rowidentifier 
, idorganisationvisibleto
, dateeventrecorded
, dateevent
, iddoneby
, idorganisationdoneat
, idmultilexproduct
, idmultilexpack
, idmultilexdmd
, nameofmedication
, datemedicationstart
, datemedicationend
, medicationdosage
, medicationquantity
, issupervised
, superviseddose
, unsuperviseddose
, isrepeatmedication
, isothermedication
, isdentalmedication
, ishospitalmedication
, idorganisationregisteredat 
, cs1.care_site_id  as care_site_id_VisibleTo   
, cs2.care_site_id  as care_site_id_DoneAt
, cs4.care_site_id  as care_site_id_RegisteredAt
#,con1.concept_id as drug_source_concept_id
#,con2.concept_id as device_source_concept_id
FROM ',sourcedb,'.tbl_SRPrimaryCareMedication srm 
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on srm.IDOrganisationVisibleTo = cs1.care_site_source_value
  left join ' ,targetdb,'.care_site cs2
 on srm.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on srm.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on srm.IDOrganisationRegisteredAt = cs4.care_site_source_value
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con1
on srm.IDMultiLexDMD = con1.concept_code and (con1.domain_id like \'Drug%\' and con1.vocabulary_id = \'dm+d\')
left JOIN yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr1
ON con1.concept_id = cr1.concept_id_1 and (cr1.relationship_id = \'Maps to\'and cr1.invalid_reason IS NULL)
left join yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept con2
on srm.IDMultiLexDMD = con2.concept_code and (con2.domain_id like \'Device%\' and con2.vocabulary_id = \'SNOMED\')
left JOIN yhcr-prd-phm-bia-core.CY_CDM_VOCAB.concept_relationship cr2
ON con2.concept_id = cr2.concept_id_1 and (cr2.relationship_id = \'Maps to\'and cr2.invalid_reason IS NULL)
where srm.person_id  = obs.person_id
and DateEvent <= obs.observation_period_end_date
and DateEvent  >= obs.observation_period_start_date', sep = "")

tb48 <-bq_project_query(project_id, sql48)
tb49 <-bq_project_query(project_id, sql49)

#Qapp_PrimaryCareFDM_SRReferralIn
#Last Modified 20220831

sql50 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateReferral = null where DateReferral = \'NULL\'', sep = "")
sql51 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateOnReferralLetter = null where DateOnReferralLetter = \'NULL\'', sep = "")
sql52 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateDischarge = null where DateDischarge = \'NULL\'', sep = "")
sql53 <-paste('Update  ',sourcedb,'.tbl_SRReferralIn 
set DateOfDecision = null where DateOfDecision = \'NULL\'', sep = "")

sql54 <-paste(' truncate table ' ,targetdb,'.tbl_SRReferralIn ', sep = "")

sql55 <-paste(' insert into ' ,targetdb,'.tbl_SRReferralIn 
(
    person_id
,src_rowidentifier
,src_idorganisationvisibleto
,src_dateeventrecorded
,src_dateevent
,src_idprofileenteredby
,src_iddoneby
,src_textualeventdoneby
,src_idorganisationdoneat
,src_category
,src_primaryreason
,src_datereferral
,src_dateonreferralletter
,src_source
,src_serviceoffered
,src_servicetype
,src_rereferral
,src_urgency
,src_idcodeprimarydiagnosis
,src_referralcode
,src_idreferrallocal
,src_reasonforservicedelay
,src_outcome
,src_defaultcontactlocation
,src_datedischargescheduled
,src_datedischarge
,src_dischargereason
,src_dischargelocation
,src_idreferrer
,src_sourcetablereferrer
,src_patientaware
,src_dateofaction
,src_activestatus
,src_referredtospeciality
,src_referredtotreatmentcode
,src_idreferredto
,src_sourcetablereferrerto
,src_referralpriority
,src_referralmedium
,src_dateofdecision
,src_daycarereferral
,src_maternityreferral
,src_commissioner
,src_clientcategory
,src_primarysupportreason
,src_idcaseload
,src_idevent
,src_idpatient
,src_idorganisation 
)
SELECT sri.person_id
, cast(RowIdentifier as string)
, IDOrganisationVisibleTo
, cast(DateEventRecorded as Date) 
, cast(DateEvent as Date) 
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, Category
, PrimaryReason
, cast(DateReferral as date) as DateReferral
, cast(DateOnReferralLetter as date) as DateOnReferralLetter
, Source
, ServiceOffered
, ServiceType
, ReReferral
, Urgency
, IDCodePrimaryDiagnosis
, ReferralCode
, IDReferralLocal
, ReasonForServiceDelay
, Outcome
, DefaultContactLocation
, DateDischargeScheduled
, cast(DateDischarge as date) as DateDischarge
, DischargeReason
, DischargeLocation
, IDReferrer
, SourceTableReferrer
, PatientAware
, DateOfAction
, ActiveStatus
, ReferredToSpeciality
, ReferredToTreatmentCode
, IDReferredTo
, SourceTableReferrerTo
, ReferralPriority
, ReferralMedium
, cast(DateOfDecision as date) as DateOfDecision
, DayCareReferral
, MaternityReferral
, Commissioner
, ClientCategory
, PrimarySupportReason
, IDCaseload
, IDEvent
, IDPatient
, IDOrganisation 
FROM ',sourcedb,'.tbl_SRReferralIn sri
, ' ,targetdb,'.observation_period obs
where  sri.person_id  = obs.person_id
and DateEvent <= obs.observation_period_end_date
and DateEvent  >= obs.observation_period_start_date', sep = "")

#Qapp_PrimaryCareFDM_SRReferralOUT
#Last Modified 2022318

sql56 <-paste(' truncate table  ' ,targetdb,'.tbl_SRReferralOUT', sep = "")

sql57 <-paste(' insert into  ' ,targetdb,'.tbl_SRReferralOUT
(
    person_id
, src_rowidentifier
, src_idorganisationvisibleto
, src_dateeventrecorded
, src_dateevent
, src_idprofileenteredby
, src_iddoneby
, src_textualeventdoneby
, src_idorganisationdoneat
, src_typeofreferral
, src_reason
, src_idprofilereferrer
, src_serviceoffered
, src_rereferral
, src_urgency
, src_primarydiagnosis
, src_recipientid
, src_recipientidtype
, src_idevent
, src_idpatient
, src_idorganisation
, src_m101140reasonforoutofareareferraladultacutementalhealth
, src_idorganisationregisteredat
, care_site_id_VisibleTo
, care_site_id_DoneAt
, care_site_id
, care_site_id_RegisteredAt
, provider_id
)
SELECT  sro.person_id
,  cast(RowIdentifier as int64)
, IDOrganisationVisibleTo
, cast(DateEventRecorded as Date)
, cast(DateEvent as Date) 
, IDProfileEnteredBy
, IDDoneBy
, TextualEventDoneBy
, IDOrganisationDoneAt
, TypeOfReferral
, Reason
, IDProfileReferrer
, ServiceOffered
, ReReferral
, Urgency
, PrimaryDiagnosis
, RecipientID
, RecipientIDType
, IDEvent
, IDPatient
, IDOrganisation
, M101140ReasonForOutOfAreaReferralAdultAcuteMentalHealth
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_care_site_id_VisibleTo
, cs2.care_site_id as care_site_id_DoneAt
, cs3.care_site_id as care_site_id
, cs4.care_site_id as  care_site_id_RegisteredAt
, null as provider_id
FROM ',sourcedb,'.tbl_SRReferralOut sro
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on sro.IDOrganisationVisibleTo = cs1.care_site_source_value
  left join ' ,targetdb,'.care_site cs2
 on sro.IDOrganisationDoneAt = cs2.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on sro.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on sro.IDOrganisationRegisteredAt = cs4.care_site_source_value
where  sro.person_id  = obs.person_id
and  dateevent <= obs.observation_period_end_date
and  dateevent  >= obs.observation_period_start_date', sep = "")


tb50 <-bq_project_query(project_id, sql50)
tb51 <-bq_project_query(project_id, sql51)
tb52 <-bq_project_query(project_id, sql52)
tb53 <-bq_project_query(project_id, sql53)
tb54 <-bq_project_query(project_id, sql54)
tb55 <-bq_project_query(project_id, sql55)
tb56 <-bq_project_query(project_id, sql56)
tb57 <-bq_project_query(project_id, sql57)

#Qapp_PrimaryCareFDM_SRVisit
#Last Modified 20220831 - JDB
sql58 <-paste(' truncate table ' ,targetdb,'.tbl_SRVisit', sep = "")

sql59 <-paste(' insert into ' ,targetdb,'.tbl_SRVisit 
(
    person_id
,src_rowidentifier
,src_idorganisationvisibleto
,src_dateeventrecorded
,src_idprofileenteredby
,src_datebooked
,src_daterequested
,src_currentstatus
,src_idprofilerequested
,src_idprofileassigned
,src_followupdetails
,src_idreferralin
,src_idpatient
,src_idorganisation
,src_duration
,src_idorganisationregisteredat
, care_site_id_VisibleTo
, care_site_id
, care_site_id_RegisteredAt
#, provider_id
)
SELECT distinct srv.person_id
, srv.RowIdentifier
, IDOrganisationVisibleTo
, DateEventRecorded
, IDProfileEnteredBy
, DateBooked
, DateRequested
, CurrentStatus
, IDProfileRequested
, IDProfileAssigned
, FollowUpDetails
, IDReferralIN
, IDPatient
, IDOrganisation
, Duration
, IDOrganisationRegisteredAt
, cs1.care_site_id as care_site_id_VisibleTo
, cs3.care_site_id as care_site_id
, cs4.care_site_id as care_site_id_RegisteredAt
#,  provider_id
FROM ',sourcedb,'.tbl_SRVisit srv
, ' ,targetdb,'.observation_period obs
 left join ' ,targetdb,'.care_site cs1
 on srv.IDOrganisationVisibleTo = cs1.care_site_source_value
 left join ' ,targetdb,'.care_site cs3
 on srv.IDOrganisation = cs3.care_site_source_value
 left join ' ,targetdb,'.care_site cs4
 on srv.IDOrganisationRegisteredAt = cs4.care_site_source_value
where  srv.person_id  = obs.person_id
and srv.DateBooked <= obs.observation_period_end_date
and srv.DateBooked >= obs.observation_period_start_date', sep = "")

tb58 <-bq_project_query(project_id, sql58)
tb59 <-bq_project_query(project_id, sql59)

#Qapp_PrimaryCareFDM_StaffMemberProfileRole
#Last Modified 20220131 - JDB

sql60 <-paste(' truncate table  ' ,targetdb,'.tbl_SRStaffMemberProfile', sep = "")

sql61 <-paste(' insert into ' ,targetdb,'.tbl_SRStaffMemberProfile 
SELECT  RowIdentifier, 
DateProfileCreated, 
IDProfileCreatedBy, 
IDStaffMemberProfileRole, 
StaffRole, DateEmploymentStart, 
DateEmploymentEnd, 
PPAID, 
GPLocalCode, 
IDStaffMember, 
IDOrganisation, 
GmpID,
cs1.care_site_id as care_site_id
FROM ',sourcedb,'.tbl_SRStaffMemberProfile smpr
left join ' ,targetdb,'.care_site cs1
 on smpr.IDOrganisation = cs1.care_site_source_value', sep = "")

tb60 <-bq_project_query(project_id, sql60)
tb61 <-bq_project_query(project_id, sql61)

#VISITBuilders
#These are added value scripts that build pseudo visit occurences and vist detail tables. 
#Finally – start the Visit Builder scripts 
#Qapp_PrimaryCareFDM_VisitBuilder_Part1
#Last modified 20220831 - JDB

sql62 <-paste(' truncate table ' ,targetdb,'.tbl_visit_builder', sep = "")

sql63 <-paste(' insert into ' ,targetdb,'.tbl_visit_builder
(
 src_rowidentifier
, src_idvisit
, src_idappointment
)
select distinct  a.src_rowidentifier 
      ,cast(a.src_idvisit as int64) 
      ,cast(a.src_idappointment as int64) 
    from ' ,targetdb,'.tbl_SRCode a
  , ' ,targetdb,'.person c
  , ' ,targetdb,'.observation_period obs
  where a.person_id = c.person_id
  and c.person_id = obs.person_id
  and a.src_dateevent <= obs.observation_period_end_date
  and a.src_dateevent >= obs.observation_period_start_date', sep = "")






sql64 <-paste('Update ' ,targetdb,'.tbl_visit_builder 
set visit_type = null 
where visit_type is not null', sep = "") 

#print (sql64) 

sql65 <-paste('Update ' ,targetdb,'.tbl_visit_builder
set visit_type= \'detail\'
where src_idvisit > 0
or src_idappointment > 0', sep = "")

sql66 <-paste('Update ' ,targetdb,'.tbl_visit_builder
set visit_type= \'occ\'
where visit_type is null ', sep = "")

#Qapp_PrimaryCareFDM_Visit_Detail_srcVisits
#Last modified 20220831 - JDB

sql67 <-paste(' truncate table ' ,targetdb,'.tbl_visit_detail', sep = "") 

sql68 <-paste(' insert into ' ,targetdb,'.tbl_visit_detail 
(      visit_detail_id 
      ,person_id
      ,src_rowid      
      ,src_table
      ,src_clinicaldataid 
      ,src_visitstartdate
      ,src_visitstartdatetime
      ,src_visitenddate
      ,src_visitenddatetime
      ,src_visitduration
      ,src_visitstartplace
      ,src_visitendplace
      ,src_visitlocation
      ,src_visitprovider
    ,src_visitdesc
    ,visit_detail_concept_id
    ,care_site_id
)
select distinct ROW_NUMBER() over (Order by c.src_rowidentifier) as visit_detail_id
  , b.person_id 
  , cast(c.src_rowidentifier as string) as src_rowid  
  , \'primary care srvisit\' as src_table
  , cast(c.src_IDVisit as string) as src_clinicaldataid
  ,cast(a.src_Daterequested as date) as visitstartdate
  ,a.src_Daterequested as visitstartdatetime
  ,cast(a.src_Daterequested as date)  as visitenddate
  ,a.src_Daterequested as visitenddatetime
  ,a.src_Duration
  ,a.src_IDOrganisation as startplace
  ,a.src_IDOrganisation as endplace
  ,a.src_IDOrganisation as visitlocation
  ,d.StaffRole as visitprovider
  ,\'primary care srvisit detail\' as visitdesc
  ,38004693 as visit_detail_concept_id
  ,cs1.care_site_id  as care_site_id
   from ' ,targetdb,'.tbl_SRVisit  a 
  inner join ' ,targetdb,'.tbl_SRPatient  b
  on a.person_id = b.person_id  
  inner join ' ,targetdb,'.tbl_visit_builder c
  on a.src_rowidentifier = c.src_IDVisit
  left join ' ,targetdb,'.tbl_SRStaffMemberProfile d
  on (a.src_IDProfileAssigned = d.IDStaffMemberProfileRole and a.src_IDOrganisation = d.IDOrganisation)
   left join ' ,targetdb,'.care_site cs1
 on a.src_IDOrganisation = cs1.care_site_source_value
  where c.visit_type = \'detail\'
  and cast(c.src_IDVisit as string)  not in 
(Select src_RowID from ' ,targetdb,'.tbl_visit_detail 
where src_Table = \'primary care srvisit\')', sep = "")

#Appends the appointments to the visit detailtable 

# Qapp_PrimaryCareFDM_Visit_Detail_srcAppointments

#Last modified 20220831 - JDB

sql69 <-paste('delete from ' ,targetdb,'.tbl_visit_detail 
where src_VisitDesc !=\'primary care srvisit detail\'', sep = "")

sql70 <-paste(' insert into ' ,targetdb,'.tbl_visit_detail 
 (     visit_detail_id 
      ,person_id
      ,src_rowid
      ,src_table
      ,src_clinicaldataid 
      ,src_visitstartdate
      ,src_visitstartdatetime
      ,src_visitenddate
      ,src_visitenddatetime
      ,src_visitduration
      ,src_visitstartplace
      ,src_visitendplace
      ,src_visitlocation
      ,src_visitprovider
      ,src_visitdesc
      ,visit_detail_concept_id
    ,care_site_id
)
Select distinct ROW_NUMBER() over (Order by c.src_rowidentifier) as visit_detail_id
      , a. person_id
      , cast(c.src_RowIdentifier as string)  as src_rowid 
  , \'primary care srappointment\' as sourcetable
  ,cast(c.src_IDAppointment as string)   as src_clinicaldataid 
     ,cast(a.src_datepatientarrival as date)
      ,cast(a.src_datepatientarrival as datetime)  as startdatetime
      ,cast(a.src_datepatientseen as date)
      ,cast(a.src_datepatientseen as datetime) as enddatetime
     ,cast(TIMESTAMP_DIFF(cast(a.src_DatePatientSeen as datetime) ,cast(a.src_DatePatientArrival as datetime) , MINUTE) as string) as duration
      ,a.src_IDOrganisation as src_visitstartplace
      ,a.src_IDOrganisation as src_visitendplace
      ,a.src_IDOrganisation as src_visitlocation 
      ,StaffRole as visitprovider
      ,case when a.src_TelephoneAppointment = \'true\' then \'telehealth\' 
      when a.src_RotaName like \'%home%visit%\' then \'home visit\'
      when a.src_RotaType like \'%home%visit%\' then \'home visit\' 
      when a.src_RotaName like \'%session%\' then \'clinic or group practice\'
      when a.src_RotaType like \'%session%\' then \'clinic or group practice\' 
      when a.src_RotaName like \'%clinic%\' then \'clinic or group practice\'
      when a.src_RotaType like \'%clinic%\' then \'clinic or group practice\' 
      when a.src_RotaName like \'%surgery%\' then \'clinic or group practice\'
      when a.src_RotaType like \'%surgery%\' then \'clinic or group practice\' 
      else a.src_RotaName end as visitdesc
      ,38004693 as visit_detail_concept_id
    ,cs1.care_site_id as care_site_id
        from ' ,targetdb,'.tbl_SRAppointment a
        inner join ' ,targetdb,'.tbl_SRPatient b
        on a.person_id = b.person_id            
        inner join ' ,targetdb,'.tbl_visit_builder c
        on a.src_RowIdentifier = c.src_IDAppointment
        left join ' ,targetdb,'.tbl_SRStaffMemberProfile d
        on cast(a.src_IDProfileClinician as int64)  = d.RowIdentifier
           left join ' ,targetdb,'.care_site cs1
 on a.src_IDOrganisation = cs1.care_site_source_value
        where a.src_DateAppointmentCancelled is null
       and a.src_DatePatientseen is not null ', sep = "")


#Qapp_PrimaryCareFDM_Visit_Occurrences_ALL
#Last modified - 20220831 - JDB

#amended to sql <-paste(' insert everything into the occurrences table 

sql71 <-paste(' truncate table ' ,targetdb,'.tbl_visit_occurrence ', sep = "")

sql72 <-paste(' insert into ' ,targetdb,'.tbl_visit_occurrence 
(      person_id
      ,visit_occurrence_id
      ,src_rowid      -- the new visit table rowidentifier
      ,src_table#
      ,src_clinicaldataid -- the actual rowidentifier from the srcode table
      ,src_visitstartdate
      ,src_visitstartdatetime
      ,src_visitenddate
      ,src_visitenddatetime
      ,src_visitduration
      ,src_visitstartplace
      ,src_visitendplace
      ,src_visitlocation
      ,care_site_id
      ,src_visitprovider
#       --,provider_id
      ,src_visitdesc
      ,visit_source_concept_id
)
select distinct a.person_id
      ,c.src_RowIdentifier   as visit_occurrence_id
      ,cast(a.src_RowIdentifier as string)  as src_rowid 
        , \'primary care srcode\' as src_table
        ,cast(a.src_RowIdentifier as string)  as src_clinicaldataid 
      ,cast(a.src_DateEvent as Date) as visitstartdate
        ,cast(a.src_DateEvent as DateTime) as visitstartdatetime
      ,cast(a.src_DateEvent as Date) as visitenddate
        ,cast(a.src_DateEvent as DateTime) as visitenddatetime
      ,\'0\' as duration
        ,a.src_IDOrganisationDoneAt as visitstartplace
        ,a.src_IDOrganisationDoneAt as visitendplace
        ,a.src_IDOrganisationDoneAt as visitlocation
        ,d.care_site_id  as care_site_id
        ,a.src_IDDoneBy as visitprovider
       # --,e.provider_id as provider_id
        ,a.src_ctv3text as visitdesc
        ,38004693 as visit_source_concept_id
   from ' ,targetdb,'.tbl_SRCode  a 
  inner join ' ,targetdb,'.person b
  on a.person_id = b.person_id 
  inner join ' ,targetdb,'.tbl_visit_builder c
  on a.src_RowIdentifier = c.src_RowIdentifier
  left join ' ,targetdb,'.care_site d
  on a.src_IDOrganisationDoneAt = d.care_site_source_value ', sep = "")


#This section builds the imosphere version of the visit_occurrence table

sql73 <-paste('drop table if exists ',targetdb,'.visit_occurrence_imo', sep = "")

sql74 <-paste('create table ',targetdb,'.visit_occurrence_imo
(
  visit_occurrence_id           INT64     not null ,
  person_id                     INT64     not null ,
  visit_concept_id              INT64     not null ,
  visit_start_DATE              DATE      not null ,
  visit_start_DATETIME          DATETIME           ,
  visit_end_DATE                DATE      not null ,
  visit_end_DATETIME            DATETIME           ,
  visit_type_concept_id         INT64     not null ,
  provider_id                   INT64              ,
  care_site_id                  STRING             ,
  visit_source_value            STRING             ,
  visit_source_concept_id       INT64              ,
  admitting_source_concept_id   INT64              ,
  admitting_source_value        STRING             ,
  discharge_to_concept_id       INT64              ,
  discharge_to_source_value     STRING             ,
  preceding_visit_occurrence_id INT64
)', sep = "")






sql75 <-paste(' insert into yhcr-prd-phm-bia-core.CY_FDM_PrimaryCare_v4.visit_occurrence_imo
(
  visit_occurrence_id,
person_id,
visit_concept_id,
visit_start_date,
visit_start_datetime,
visit_end_date,
visit_type_concept_id,
care_site_id
)
select RowIdentifier
,vis.person_id
,9202 as visit_concept_id
,cast(DateBooked as date)  
,DateBooked
,cast(DateBooked as Date) 
,32817 as visit_type_concept_id
,IDOrganisation
FROM ',sourcedb,'.tbl_SRVisit vis
 , ' ,targetdb,'.observation_period obs
  where vis.person_id = obs.person_id
  and vis.DateBooked <= obs.observation_period_end_date
  and vis.DateBooked >= obs.observation_period_start_date', sep = "")

#New section for configured ListOption

sql76 <-paste('drop table if exists ' ,targetdb,'.tbl_SRConfiguredListOption', sep = "")
sql77 <-paste('create table ' ,targetdb,'.tbl_SRConfiguredListOption as select cast(RowIdentifier as int64) as src_RowIdentifier
      ,cast(ConfiguredList as int64) as ConfiguredList
      ,ConfiguredListOption
      ,CDSCode
      ,CAMHSCode
      ,MHSDSCode from ',sourcedb,'.tbl_SRConfiguredListOption', sep = "")



#Now run the SQL 

tb62 <-bq_project_query(project_id, sql62)
tb63 <-bq_project_query(project_id, sql63)
tb64 <-bq_project_query(project_id, sql64)
tb65 <-bq_project_query(project_id, sql65)
tb66 <-bq_project_query(project_id, sql66)
tb67 <-bq_project_query(project_id, sql67)
tb68 <-bq_project_query(project_id, sql68)
tb69 <-bq_project_query(project_id, sql69)
tb70 <-bq_project_query(project_id, sql70)
tb71 <-bq_project_query(project_id, sql71)
tb72 <-bq_project_query(project_id, sql72)
tb73 <-bq_project_query(project_id, sql73)
tb74 <-bq_project_query(project_id, sql74)
tb75 <-bq_project_query(project_id, sql75)
tb76 <-bq_project_query(project_id, sql76)
tb77 <-bq_project_query(project_id, sql77)
